In [7]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [8]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
from src.models.EMOS_global.EMOS_global_load_score import *
from src.models.EMOS_local.EMOS_local_load_score import *
from src.models.EMOS_local.EMOS_local_load_model import *
from src.models.EMOS_global.EMOS_global_load_model import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *
from src.models.EMOS_local.EMOS_local_train import *

<IPython.core.display.Javascript object>

### 1. Load Scores

In [9]:
EMOS_local_t2m_scores = EMOS_local_load_score("t2m")
EMOS_global_t2m_scores = EMOS_global_load_score("ws10")
CRPS_global_ws10_scores = crps_load_lead_lat_lon("ws10")

<IPython.core.display.Javascript object>

In [5]:
len(EMOS_local_t2m_scores)

0

<IPython.core.display.Javascript object>

In [14]:
EMOS_local_t2m_scores[0].shape

IndexError: list index out of range

<IPython.core.display.Javascript object>

In [15]:
EMOS_local_t2m_scores[0]

IndexError: list index out of range

<IPython.core.display.Javascript object>

In [7]:
for i in range(len(EMOS_local_t2m_scores)):
    print(np.mean(EMOS_local_t2m_scores[i]))

327.2017108996968
328.0493000664214
332.48294152838247
330.6740324498626
326.73660801611754
326.70713238555567
329.32984385920594
323.13521994302386
325.1597666886211
328.6905720557139
326.0973693368299
328.52279513621494
321.9730010965976
329.28202453296285
328.9977007887049
322.3121892305651
324.76438635533987
332.51399195103966
329.1149755829886
325.0247790620081
325.27671443053555
327.84375494602887
328.1380074302177
325.2698518401066
326.17470938007926
329.4066426815335
328.1277118818608
324.0014038933928
324.11201905947763
328.7127176491239
0.0


<IPython.core.display.Javascript object>

In [42]:
for i in range(len(EMOS_global_t2m_scores)):
    print(np.mean(EMOS_global_t2m_scores[i]))

0.46879795
0.54816717
0.5383585
0.5816508
0.5986497
0.69715935
0.69927776
0.7526292
0.79125553
0.89870316
0.8913976
0.93647575
0.9739682
1.0816439
1.0613875
1.0941627
1.1224575
1.2188288
1.1861817
1.2084353
1.2331907
1.3312402
1.2901597
1.3045237
1.3217683
1.4142808
1.3631004
1.37057
1.382329
1.4722959
1.4110365


<IPython.core.display.Javascript object>

In [43]:
for i in range(len(CRPS_global_ws10_scores)):
    print(np.mean(CRPS_global_ws10_scores[i]))

0.47082883
0.5626273
0.5660685
0.62446654
0.62736267
0.74193853
0.7447009
0.8054526
0.8340255
0.9581987
0.9486295
0.99771065
1.0248152
1.1472718
1.1209658
1.1554667
1.1716784
1.2803926
1.2406598
1.2650195
1.2812319
1.3930285
1.3470749
1.3625422
1.3708637
1.4767841
1.4213701
1.429318
1.4322772
1.5369276
1.469894


<IPython.core.display.Javascript object>

### 3. Hyperparameter tuning:

In [1]:
from src.models.EMOS_local.EMOS_local_hyper_tune import *

2023-06-01 14:10:45.345108: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-01 14:10:45.345130: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
var_num = 5
lead_time = 30
# Perhaps tunable
epochs = [30]
# To tune I guess
lrs = [0.1, 0.01, 0.001]
batch_sizes = [32, 64, 128, 256]
optimizers = ["Adam"]
validation_split = 0.2

In [3]:
EMOS_local_best_params, EMOS_local_best_score = EMOS_local_hyper_tune(
    var_num,
    lead_time,
    batch_sizes=batch_sizes,
    epochs=epochs,
    lrs=lrs,
    optimizers=optimizers,
)

2023-06-01 14:10:47.836951: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-01 14:10:47.837040: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-01 14:10:47.837100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-01 14:10:47.837159: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file:

Best validation score = 2.1412482261657715
Best validation score = 2.198239326477051
Best validation score = 2.1896729469299316
Best validation score = 2.2473511695861816
Best validation score = 2.2649879455566406
Best validation score = 2.1744801998138428
Best validation score = 2.146947145462036
Best validation score = 2.120931386947632
Best validation score = 2.182908535003662
Best validation score = 2.39786958694458
Best validation score = 0.377621054649353
Best validation score = 0.786067008972168
Best validation score = 0.9189858436584473
Best validation score = 2.2094333171844482
Best validation score = 2.2057976722717285
Best validation score = 2.123753070831299
Best validation score = 2.1673576831817627
Best validation score = 2.1808555126190186
Best validation score = 2.0553081035614014
Best validation score = 1.8677902221679688
Best validation score = 1.9741367101669312
Best validation score = 2.2584786415100098
Best validation score = 2.0945870876312256
Best validation scor

Best validation score = 1.9733322858810425
Best validation score = 1.2305506467819214
Best validation score = 0.7273597717285156
Best validation score = 1.963649034500122
Best validation score = 2.007549285888672
Best validation score = 2.0289788246154785
Best validation score = 2.0151307582855225
Best validation score = 0.9502220153808594
Best validation score = 1.6453299522399902
Best validation score = 0.9628456830978394
Best validation score = 1.85275399684906
Best validation score = 1.9683732986450195
Best validation score = 2.1514148712158203
Best validation score = 2.0473825931549072
Best validation score = 1.1793277263641357
Best validation score = 1.5565253496170044
Best validation score = 2.075993061065674
Best validation score = 2.056767463684082
Best validation score = 1.9672627449035645
Best validation score = 1.4471476078033447
Best validation score = 0.8846967220306396
Best validation score = 2.0376577377319336
Best validation score = 1.0048843622207642
Best validation s

Best validation score = 2.0269134044647217
Best validation score = 1.0570521354675293
Best validation score = 0.993385374546051
Best validation score = 0.7834852933883667
Best validation score = 0.6924489736557007
Best validation score = 0.7410463690757751
Best validation score = 1.9240199327468872
Best validation score = 1.8130255937576294
Best validation score = 1.8264524936676025
Best validation score = 1.8110324144363403
Best validation score = 1.7575132846832275
Best validation score = 1.7982561588287354
Best validation score = 1.0557761192321777
Best validation score = 0.9171767830848694
Best validation score = 0.8410201668739319
Best validation score = 0.8425914645195007
Best validation score = 0.6077020168304443
Best validation score = 0.45185837149620056
Best validation score = 0.33025556802749634
Best validation score = 1.6882166862487793
Best validation score = 1.712646245956421
Best validation score = 1.7811508178710938
Best validation score = 1.8245346546173096
Best valida

Best validation score = 0.6570183038711548
Best validation score = 0.8308524489402771
Best validation score = 0.6916170120239258
Best validation score = 0.46768707036972046
Best validation score = 0.40890195965766907
Best validation score = 2.140364408493042
Best validation score = 1.8076423406600952
Best validation score = 1.6839535236358643
Best validation score = 0.5857251286506653
Best validation score = 1.6892305612564087
Best validation score = 1.6137126684188843
Best validation score = 1.7334200143814087
Best validation score = 1.9326963424682617
Best validation score = 0.6255188584327698
Best validation score = 0.6055351495742798
Best validation score = 0.7077805995941162
Best validation score = 0.7192727327346802
Best validation score = 1.621721863746643
Best validation score = 1.6916769742965698
Best validation score = 1.6525274515151978
Best validation score = 1.6219615936279297
Best validation score = 1.4750159978866577
Best validation score = 1.5019611120224
Best validatio

...

Best validation score = 0.5592237710952759
Best validation score = 0.754060685634613
Best validation score = 2.13071346282959
Best validation score = 2.195274591445923
Best validation score = 2.101830005645752
Best validation score = 2.0673069953918457
Best validation score = 2.051156759262085
Best validation score = 0.7899904251098633
Best validation score = 1.2527841329574585
Best validation score = 2.018792152404785
Best validation score = 1.9862149953842163
Best validation score = 2.154189348220825
Best validation score = 1.9603663682937622
Best validation score = 1.2206165790557861
Best validation score = 0.7235593199729919
Best validation score = 1.9660253524780273
Best validation score = 2.0057058334350586
Best validation score = 2.0265913009643555
Best validation score = 2.02046275138855
Best validation score = 0.948548436164856
Best validation score = 1.6466253995895386
Best validation score = 0.9617072939872742
Best validation score = 1.8548486232757568
Best validation score 

Best validation score = 1.9117252826690674
Best validation score = 1.1705331802368164
Best validation score = 0.8767057061195374
Best validation score = 0.8183183073997498
Best validation score = 0.7586665153503418
Best validation score = 2.0586929321289062
Best validation score = 1.8995760679244995
Best validation score = 1.9540175199508667
Best validation score = 1.8337056636810303
Best validation score = 1.7560464143753052
Best validation score = 1.553394079208374
Best validation score = 1.8292129039764404
Best validation score = 1.9635452032089233
Best validation score = 1.0456230640411377
Best validation score = 0.9452025890350342
Best validation score = 0.7557070851325989
Best validation score = 0.6914377212524414
Best validation score = 0.7464480400085449
Best validation score = 1.9284343719482422
Best validation score = 1.8196001052856445
Best validation score = 1.8230036497116089
Best validation score = 1.7844913005828857
Best validation score = 1.7747610807418823
Best validat

Best validation score = 1.7061485052108765
Best validation score = 4.218138694763184
Best validation score = 0.7744595408439636
Best validation score = 0.8121989965438843
Best validation score = 1.185447096824646
Best validation score = 0.3569466769695282
Best validation score = 0.5987544655799866
Best validation score = 1.8358235359191895
Best validation score = 2.142197370529175
Best validation score = 1.7918137311935425
Best validation score = 4.688724994659424
Best validation score = 2.344587802886963
Best validation score = 1.0430797338485718
Best validation score = 0.8488062620162964
Best validation score = 1.0881627798080444
Best validation score = 0.48207587003707886
Best validation score = 0.4327734410762787
Best validation score = 2.522768974304199
Best validation score = 2.0331740379333496
Best validation score = 1.7937487363815308
Best validation score = 0.612754225730896
Best validation score = 1.9309124946594238
Best validation score = 2.0753557682037354
Best validation s

Best validation score = 1.6877423524856567
Best validation score = 1.8186105489730835
Best validation score = 0.8140602111816406
Best validation score = 0.7034887671470642
Best validation score = 0.7969633340835571
Best validation score = 1.6033076047897339
Best validation score = 1.6328459978103638
Best validation score = 1.517382264137268
Best validation score = 1.4083954095840454
Best validation score = 1.666232705116272
Best validation score = 1.5953303575515747
Best validation score = 1.4626811742782593
Best validation score = 1.432389259338379
Best validation score = 1.5469298362731934
Best validation score = 1.492417812347412
Best validation score = 1.2904393672943115
Best validation score = 0.3523804843425751
Best validation score = 0.5514472723007202
Best validation score = 0.5953184962272644
Best validation score = 1.0982465744018555
Best validation score = 1.3360044956207275
Best validation score = 0.8084407448768616
Best validation score = 0.7346295714378357
Best validation

Best validation score = 1.0548039674758911
Best validation score = 2.322946786880493
Best validation score = 2.9594523906707764
Best validation score = 4.212508201599121
Best validation score = 2.7054433822631836
Best validation score = 2.900843381881714
Best validation score = 2.2798571586608887
Best validation score = 2.6244821548461914
Best validation score = 3.083235025405884
Best validation score = 2.812246799468994
Best validation score = 2.108271360397339
Best validation score = 0.9953714609146118
Best validation score = 0.5745906829833984
Best validation score = 0.794430136680603
Best validation score = 2.8674256801605225
Best validation score = 2.2015721797943115
Best validation score = 2.967278480529785
Best validation score = 3.160768747329712
Best validation score = 2.4226248264312744
Best validation score = 1.2204831838607788
Best validation score = 1.425706148147583
Best validation score = 2.0607311725616455
Best validation score = 2.7662811279296875
Best validation score

In [4]:
print(f'Best score {EMOS_local_best_score} and best params {EMOS_local_best_params}')

Best score 1.9662694050333438 and best params (256, 30, 0.001, 'Adam')


In [10]:
EMOS_global_t2m_scores[lead_time].mean()

1.4110365

<IPython.core.display.Javascript object>

### 3. Trying to load models:

In [ ]:
EMOS_local_t2m_0 = EMOS_local_load_model(2, 0)

In [24]:
lat = 2
lon = 2
lead_time = 0

<IPython.core.display.Javascript object>

In [28]:
test_var_denormed = ldpd.load_data_all_train_proc_denorm()[2]

<IPython.core.display.Javascript object>

In [29]:
X_test_var_denormed = test_var_denormed[
    list(test_var_denormed.data_vars.keys())[0]
].isel(lead_time=lead_time, lat=lat, lon=lon)
y_test_var_denormed = test_var_denormed[
    list(test_var_denormed.data_vars.keys())[1]
].isel(lead_time=lead_time, lat=lat, lon=lon)

<IPython.core.display.Javascript object>

In [30]:
EMOS_local_t2m_0[lat][lon].evaluate(
    [
        X_test_var_denormed.isel(mean_std=0).values.flatten(),
        X_test_var_denormed.isel(mean_std=1).values.flatten(),
    ],
    y_test_var_denormed.values.flatten(),
)

45/45 [==============================] - 0s 927us/step - loss: 402.3280


402.3279724121094

<IPython.core.display.Javascript object>